# Kriging Playground

### Prepare Workspace

In [6]:
# Import system libraries
import os
import sys
import warnings
warnings.filterwarnings("ignore")

# Import data manipulation librariaes
import pandas as pd
import numpy as np
import xarray as xr
import rioxarray
import rasterio.mask

# Import statistical libraries
from pykrige.ok import OrdinaryKriging

# Set working directory
os.chdir('/Users/jessicarapson/Documents/GitHub/water-supply-forecast')

### Build Kriging Model

In [ ]:
# Load raster data (snow and elevation)
snow_raster = xr.open_rasterio('path_to_snow_raster.tif')  # Replace with your snow raster path
elevation_raster = xr.open_rasterio('path_to_elevation_raster.tif')  # Replace with your elevation raster path

# Read the raster data as numpy arrays
snow_array = snow_raster.values[0]  # Assuming single band data
elevation_array = elevation_raster.values[0]  # Assuming single band data

# Assuming you have streamflow measurements (x, y, streamflow_value) at known locations
# Replace the following placeholders with your actual streamflow measurements
streamflow_measurements = [
    {'x': 123, 'y': 456, 'value': 10},
    {'x': 234, 'y': 567, 'value': 20},
    # Add more measurements as needed
]

# Extract coordinates and values of streamflow measurements
measurements_coords = np.array([[m['x'], m['y']] for m in streamflow_measurements])
measurements_values = np.array([m['value'] for m in streamflow_measurements])

# Create a grid of coordinates covering the area of interest using rasterio mask
with rasterio.open('path_to_snow_raster.tif') as src:
    out_image, out_transform = rasterio.mask.mask(src, [src.bounds], crop=True)
    out_meta = src.meta.copy()

    x_coords = np.linspace(out_meta['transform'][2], out_meta['transform'][2] + out_meta['width'] * out_meta['transform'][0], out_meta['width'])
    y_coords = np.linspace(out_meta['transform'][5], out_meta['transform'][5] + out_meta['height'] * out_meta['transform'][4], out_meta['height'])
    x_mesh, y_mesh = np.meshgrid(x_coords, y_coords)
    raster_points = np.column_stack([x_mesh.ravel(), y_mesh.ravel()])

# Fit empirical variogram
ok = OrdinaryKriging(
    xi=measurements_coords[:, 0],
    yi=measurements_coords[:, 1],
    zi=measurements_values,
    xk=raster_points[:, 0],
    yk=raster_points[:, 1],
)

variogram_model, variogram_model_parameters = ok.execute('variogram_model')

# Perform Ordinary Kriging
kriging_result, kriging_variance = ok.execute('grid')

# Get the predicted streamflow at a specific point
desired_point = np.array([[desired_x, desired_y]])  # Replace with the desired point coordinates
kriging_point_result, kriging_point_variance = ok.execute('points', desired_point)

print("Predicted Streamflow at Desired Point:", kriging_point_result)
